**Important! Before you run the codes in this notebook, you firstly need to download the 'InstallLog.txt' and 'Accidents2019.csv' from Blackboard--Dataset folder and upload it to Kaggle.**

In [2]:
# install PySpark 
#http://spark.apache.org/docs/latest/api/python/index.html

!pip install pyspark==3.2


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.3/281.3 MB 4.0 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.8/198.8 kB 12.0 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805913 sha256=32af64c776020622f6155e93a49b31c0feaca1850a57925ca60b043eb2d3bc2b
  Stored in directory: /root/.cache/pip/wheels/e7/7a/38/336d4a32e61c410ce83bf854593a1955d2d987060c0f26d373
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [2]:
# start spark sessnon and configureation
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession, SQLContext
spark = SparkSession.builder.master("local[2]").appName("rdd").getOrCreate()
sc = spark.sparkContext

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/04/26 12:56:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


**You might need to change the file path in sc.textfile() accordingly**

In [3]:
# file is available in blackboard under week 2
# create RDD


logData = sc.textFile("/kaggle/input/install-log/InstallLog(1).txt")

In [4]:
logData.first()

'02/08/20 22:33:12:735 | [INFO] | 2336 | HDSetup | HDSetup | AppContext |  | HDSetup | 23620 | ************************* START Adobe Setup *************************'

In [5]:
logData.take(5)

['02/08/20 22:33:12:735 | [INFO] | 2336 | HDSetup | HDSetup | AppContext |  | HDSetup | 23620 | ************************* START Adobe Setup *************************',
 '02/08/20 22:33:12:736 | [INFO] | 2336 | HDSetup | HDSetup | AppContext |  | HDSetup | 23620 | Adobe Setup Version: 5.0.0.354',
 '02/08/20 22:33:12:736 | [INFO] | 2336 | HDSetup | OOBEUtils | CommandLineParser |  | OOBEUtils | 23620 | Parsing the command line provided. Number of command line arguments is 2',
 '02/08/20 22:33:18:553 | [INFO] | 2336 | HDSetup | HDPIM | HDPIMSessionManager |  | HDPIM | 23332 | Inside hdpimCreateSession',
 "02/08/20 22:33:18:554 | [INFO] | 2336 | HDSetup | HDPIM | HDPIMSessionManager |  | HDPIM | 23332 | Entering into 'createHDPIMSession'"]

In [6]:
logData.count()

361

In [8]:
setups = logData.filter(lambda line: "HDSetup" in line)

In [9]:
setups.take(3)

['02/08/20 22:33:12:735 | [INFO] | 2336 | HDSetup | HDSetup | AppContext |  | HDSetup | 23620 | ************************* START Adobe Setup *************************',
 '02/08/20 22:33:12:736 | [INFO] | 2336 | HDSetup | HDSetup | AppContext |  | HDSetup | 23620 | Adobe Setup Version: 5.0.0.354',
 '02/08/20 22:33:12:736 | [INFO] | 2336 | HDSetup | OOBEUtils | CommandLineParser |  | OOBEUtils | 23620 | Parsing the command line provided. Number of command line arguments is 2']

In [10]:
setups.count()

301

In [15]:
# create a simple key value pair RDD 

kvpairs = sc.parallelize([('city', 'Leicester'),('county', 'Leicestershire'),('country', 'UK')])


In [16]:
# return all the RDD's contents

kvpairs.collect()

[('city', 'Leicester'), ('county', 'Leicestershire'), ('country', 'UK')]

In [17]:
# Return all the RDD's keys

kvpairs.keys().collect()

['city', 'county', 'country']

In [18]:
# Return all the RDD's values

kvpairs.values().collect()

['Leicester', 'Leicestershire', 'UK']

In [19]:
# Create an RDD using parallelize

locations = sc.parallelize([('Leicester','UK', 1)
, ('Lagos','Nigeria', 2)
,('NY', 'USA', 3)])


In [20]:
locations.collect()

[('Leicester', 'UK', 1), ('Lagos', 'Nigeria', 2), ('NY', 'USA', 3)]

In [21]:
# create a new RDD from the locations RDD, as a key value pair with the key being the data in the third positon (0,1,"2") of locations 

bylocationNo = locations.keyBy(lambda x: x[2])

In [17]:
# return all the values in the RDD 

bylocationNo.collect()

[(1, ('Leicester', 'UK', 1)),
 (2, ('Lagos', 'Nigeria', 2)),
 (3, ('NY', 'USA', 3))]

In [18]:
# Create a new RDD starting at 0 and going to 100 in steps of 1, across 2 partitions

newRDD = sc.range(0,100,1,2)

In [19]:
newRDD.take(10)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [20]:
newRDD.getNumPartitions()

2

In [ ]:
###################  DATAFRAMES ##############################################

In [21]:
# define SQLContext
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)


/opt/conda/lib/python3.7/site-packages/pyspark/sql/context.py:79: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


In [22]:
# create a RDD using parallelize

newRDD = sc.parallelize([('Mark', 32), ('James', 56)])

In [23]:
# return entire contents of the RDD

newRDD.collect()

[('Mark', 32), ('James', 56)]

In [24]:
# Create a dataframe from the RDD and then return all it's contents

sqlContext.createDataFrame(newRDD).collect()

[Row(_1='Mark', _2=32), Row(_1='James', _2=56)]

In [25]:
# Create a new data frame df

df= sqlContext.createDataFrame(newRDD)

In [26]:
# Have a look at the new dataframe's schema (implicitly defined)

df.printSchema()

root
 |-- _1: string (nullable = true)
 |-- _2: long (nullable = true)



In [27]:
# Load a text file and convert each line to a Row
# The Accidents2019.csv file is in black board under Dataset folder

path = "/kaggle/input/accidents2019"


In [28]:
# create the RDD

data = sc.textFile(path)

In [29]:
# return the first five items in the RDD

data.take(5)

['Accident_Index,Location_Easting_OSGR,Location_Northing_OSGR,Longitude,Latitude,Police_Force,Accident_Severity,Number_of_Vehicles,Number_of_Casualties,Date,Day_of_Week,Time,Local_Authority_(District),Local_Authority_(Highway),1st_Road_Class,1st_Road_Number,Road_Type,Speed_limit,Junction_Detail,Junction_Control,2nd_Road_Class,2nd_Road_Number,Pedestrian_Crossing-Human_Control,Pedestrian_Crossing-Physical_Facilities,Light_Conditions,Weather_Conditions,Road_Surface_Conditions,Special_Conditions_at_Site,Carriageway_Hazards,Urban_or_Rural_Area,Did_Police_Officer_Attend_Scene_of_Accident,LSOA_of_Accident_Location',
 '2019010128300,528218,180407,-0.153842,51.508057,1,3,2,3,18/02/2019,2,17:50,1,E09000033,3,4202,1,30,1,2,3,4202,0,5,1,1,1,0,0,1,3,E01004762',
 '2019010152270,530219,172463,-0.127949,51.436208,1,3,2,1,15/01/2019,3,21:45,9,E09000022,3,23,2,30,0,-1,-1,0,-1,-1,4,1,1,0,0,1,3,E01003117',
 '2019010155191,530222,182543,-0.124193,51.526795,1,3,2,1,01/01/2019,3,01:50,2,E09000007,4,504,6,30,

In [30]:
# create an RDD from data RDD and split the contents by comma - as it is a comma delimited file

parts = data.map(lambda l: l.split(","))

In [31]:
parts.take(5)

[['Accident_Index',
  'Location_Easting_OSGR',
  'Location_Northing_OSGR',
  'Longitude',
  'Latitude',
  'Police_Force',
  'Accident_Severity',
  'Number_of_Vehicles',
  'Number_of_Casualties',
  'Date',
  'Day_of_Week',
  'Time',
  'Local_Authority_(District)',
  'Local_Authority_(Highway)',
  '1st_Road_Class',
  '1st_Road_Number',
  'Road_Type',
  'Speed_limit',
  'Junction_Detail',
  'Junction_Control',
  '2nd_Road_Class',
  '2nd_Road_Number',
  'Pedestrian_Crossing-Human_Control',
  'Pedestrian_Crossing-Physical_Facilities',
  'Light_Conditions',
  'Weather_Conditions',
  'Road_Surface_Conditions',
  'Special_Conditions_at_Site',
  'Carriageway_Hazards',
  'Urban_or_Rural_Area',
  'Did_Police_Officer_Attend_Scene_of_Accident',
  'LSOA_of_Accident_Location'],
 ['2019010128300',
  '528218',
  '180407',
  '-0.153842',
  '51.508057',
  '1',
  '3',
  '2',
  '3',
  '18/02/2019',
  '2',
  '17:50',
  '1',
  'E09000033',
  '3',
  '4202',
  '1',
  '30',
  '1',
  '2',
  '3',
  '4202',
  '0',

In [32]:
# create a DataFrame (implicitly define the schema)

accidents = sqlContext.createDataFrame(parts)

In [33]:
accidents

DataFrame[_1: string, _2: string, _3: string, _4: string, _5: string, _6: string, _7: string, _8: string, _9: string, _10: string, _11: string, _12: string, _13: string, _14: string, _15: string, _16: string, _17: string, _18: string, _19: string, _20: string, _21: string, _22: string, _23: string, _24: string, _25: string, _26: string, _27: string, _28: string, _29: string, _30: string, _31: string, _32: string]

In [34]:
accidents.printSchema()

root
 |-- _1: string (nullable = true)
 |-- _2: string (nullable = true)
 |-- _3: string (nullable = true)
 |-- _4: string (nullable = true)
 |-- _5: string (nullable = true)
 |-- _6: string (nullable = true)
 |-- _7: string (nullable = true)
 |-- _8: string (nullable = true)
 |-- _9: string (nullable = true)
 |-- _10: string (nullable = true)
 |-- _11: string (nullable = true)
 |-- _12: string (nullable = true)
 |-- _13: string (nullable = true)
 |-- _14: string (nullable = true)
 |-- _15: string (nullable = true)
 |-- _16: string (nullable = true)
 |-- _17: string (nullable = true)
 |-- _18: string (nullable = true)
 |-- _19: string (nullable = true)
 |-- _20: string (nullable = true)
 |-- _21: string (nullable = true)
 |-- _22: string (nullable = true)
 |-- _23: string (nullable = true)
 |-- _24: string (nullable = true)
 |-- _25: string (nullable = true)
 |-- _26: string (nullable = true)
 |-- _27: string (nullable = true)
 |-- _28: string (nullable = true)
 |-- _29: string (nullab

In [35]:
# Let's explicitly define the schema

# import library for StructType

from pyspark.sql.types import *

In [37]:
# Explicitly define a schema

myschema = StructType([ \
StructField('Accident_Index', IntegerType(), True), \
StructField('Location_Easting_OSGR', StringType(), True), \
StructField('Location_Northing_OSGR', StringType(), True)])



In [38]:
myschema

StructType(List(StructField(Accident_Index,IntegerType,true),StructField(Location_Easting_OSGR,StringType,true),StructField(Location_Northing_OSGR,StringType,true)))

In [39]:
# return first item in the data RDD

data.first()

'Accident_Index,Location_Easting_OSGR,Location_Northing_OSGR,Longitude,Latitude,Police_Force,Accident_Severity,Number_of_Vehicles,Number_of_Casualties,Date,Day_of_Week,Time,Local_Authority_(District),Local_Authority_(Highway),1st_Road_Class,1st_Road_Number,Road_Type,Speed_limit,Junction_Detail,Junction_Control,2nd_Road_Class,2nd_Road_Number,Pedestrian_Crossing-Human_Control,Pedestrian_Crossing-Physical_Facilities,Light_Conditions,Weather_Conditions,Road_Surface_Conditions,Special_Conditions_at_Site,Carriageway_Hazards,Urban_or_Rural_Area,Did_Police_Officer_Attend_Scene_of_Accident,LSOA_of_Accident_Location'

In [40]:
# create new RDD from the items in position 0, 1 and 2

rdd_expl = data.map(lambda x: x.split(',')).map(lambda x: (str(x[0]), str(x[1]), str(x[2])))

In [40]:
rdd_expl.take(5)

[('Accident_Index', 'Location_Easting_OSGR', 'Location_Northing_OSGR'),
 ('2019010128300', '528218', '180407'),
 ('2019010152270', '530219', '172463'),
 ('2019010155191', '530222', '182543'),
 ('2019010155192', '525531', '184605')]

In [41]:
# Create a data frame using the explicitly defined schema

df_expl = sqlContext.createDataFrame(rdd_expl, myschema)

In [42]:
df_expl.printSchema()

root
 |-- Accident_Index: integer (nullable = true)
 |-- Location_Easting_OSGR: string (nullable = true)
 |-- Location_Northing_OSGR: string (nullable = true)



In [43]:
df_expl

DataFrame[Accident_Index: int, Location_Easting_OSGR: string, Location_Northing_OSGR: string]